In [1]:
# importing the libraries
import numpy as np
import peakutils
import syntheticdata
import threegaussians
import lorentzian
from peakutils.plot import plot as pplot
from matplotlib import pyplot as plt
%matplotlib inline
from scipy.optimize import curve_fit
from scipy import interpolate
from astropy.modeling import models, fitting
import pandas as pd

In [15]:
def Earth_Peakutils(nm_array, timedelay,threshold,min_dist):
    import numpy
    import matplotlib.pyplot as plt
    from pyearth import Earth
    
    """
     ============================================
     Plotting derivatives of simple sine function
    ============================================

     A simple example plotting a fit of the sine function
    and the derivatives computed by Earth.
    
    Notes
    -----   
    generates a denoise curve from the TA data
    Parameters
    ----------
        nm_array: wavelength array
        timedelay: time delay array
        noise_coefficient: the noise coefficients that user want to generate
    Returns
    -------
        a smoothing curve from the original noise curve   
    """
    # Create some fake data
    # generate some noisy data from syntheticdata:
    np.random.seed(1729)
    y_noise = 0.1 * np.random.normal(size=nm_array.size)
    ydata = timedelay + y_noise
    
   # Fit an Earth model
    model = Earth(max_degree=2, minspan_alpha=.5, smooth=True)
    model.fit(nm_array, ydata)
    
   # Get the predicted values and derivatives
    y_hat = model.predict(nm_array)
   
    # use peakutils to find peak indexs
    peak_indices_true = peakutils.indexes(timedelay, thres=threshold, min_dist=min_dist)
    peak_indices_smooth = peakutils.indexes(y_hat, thres=threshold, min_dist=min_dist)
    
    return peak_indices_true,peak_indices_smooth
    

In [3]:
def astropy_Peakutils(nm_array,timedelay,gg_init,threshold,min_dist):
    # Generate fake data
    np.random.seed(42)
    ydata = timedelay + 0.1*np.random.normal(size=nm_array.size)
    # Now to fit the data create a new superposition with initial
    # guesses for the parameters:
    fitter = fitting.SLSQPLSQFitter()
    gg_fit = fitter(gg_init, nm_array, ydata)
    # use peakutils to find peak
    peak_indices_true = peakutils.indexes(timedelay, thres=threshold, min_dist=min_dist)
    peak_indices_smooth = peakutils.indexes(gg_fit(nm_array), thres=threshold, min_dist=min_dist)
  
    return peak_indices_true,peak_indices_smooth

In [14]:
def astropy_Smoothing(nm_array,timedelay,noise_coefficient,gg_init):
    # Generate fake data
    np.random.seed(42)
    ydata = timedelay + noise_coefficient*np.random.normal(size=nm_array.size)
    # Now to fit the data create a new superposition with initial
    # guesses for the parameters:
    fitter = fitting.SLSQPLSQFitter()
    gg_fit = fitter(gg_init, nm_array, ydata)
  
    return gg_fit(nm_array)

In [17]:
def Earth_Smoothing(nm_array, y_array,noise_coefficient):        
    """
    ============================================
     Plotting derivatives of simple sine function
    ============================================

     A simple example plotting a fit of the sine function
    and the derivatives computed by Earth.
    
    Notes
    -----   
    generates a denoise curve from the TA data
    Parameters
    ----------
        nm_array: wavelength array
        timedelay: time delay array
        noise_coefficient: the noise coefficients that user want to generate
    Returns
    -------
        a smoothing curve from the original noise curve   
    """
    from pyearth import Earth
   # Fit an Earth model
    model = Earth(smooth=True)
    np.random.seed(42)
    ydata = y_array + noise_coefficient*np.random.normal(size=nm_array.size)
    model.fit(nm_array, ydata)

   # Print the model
    #print(model.trace())
    #print(model.summary())

   # Get the predicted values and derivatives
    y_hat = model.predict(nm_array)
    
    return  y_hat

# Get noise data and smoothing data dataframe

* astropy

In [148]:
def astropy_smooth_matrix(nm_array,data_matrix,noise_coefficient,gg_init):
    num_array = np.shape(data_matrix)[0]
    
    smooth_matx = pd.DataFrame(np.empty((num_array,1)), columns = ['a'])
    noise_matx = pd.DataFrame(np.empty((num_array,1)), columns = ['a'])
    
    for i in range(num_array):
        data_array = data_matrix[:, i]
        
        # get noise and smooth list
        noise_array = add_noise(nm_array, data_array, noise_coefficient).tolist()
        smooth_array = astropy_Smoothing(nm_array,data_array,noise_coefficient,gg_init).tolist()
        
        # get noise dataframe
        DF = pd.DataFrame(noise_array,columns = [i])
        noise_matx = noise_matx.join(DF)
        
        # get smooth dataframe
        df = pd.DataFrame(smooth_array,columns = [i])
        smooth_matx = smooth_matx.join(df)
        
    # drop the first columns  
    noise_matx = noise_matx.drop(columns='a')
    smooth_matx = smooth_matx.drop(columns='a')
        
    return noise_matx, smooth_matx

In [149]:
noise_matx, smooth_matx = astropy_smooth_matrix(datanm,dataz_matx,0.1,gg_init)

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.7495624261835365
            Iterations: 35
            Function evaluations: 286
            Gradient evaluations: 35
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.749416522632002
            Iterations: 26
            Function evaluations: 213
            Gradient evaluations: 26
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.749433287148435
            Iterations: 35
            Function evaluations: 286
            Gradient evaluations: 35
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.749375198981689
            Iterations: 36
            Function evaluations: 293
            Gradient evaluations: 36
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.749431472957076
            Iterations: 36
            Functio

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.749372368294726
            Iterations: 36
            Function evaluations: 294
            Gradient evaluations: 36
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.749402007284543
            Iterations: 37
            Function evaluations: 301
            Gradient evaluations: 37
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.749120628475032
            Iterations: 37
            Function evaluations: 301
            Gradient evaluations: 37
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.749316680218414
            Iterations: 38
            Function evaluations: 309
            Gradient evaluations: 38
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.749372267100967
            Iterations: 38
            Function

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.748982964647777
            Iterations: 42
            Function evaluations: 341
            Gradient evaluations: 42
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.749130093999115
            Iterations: 42
            Function evaluations: 341
            Gradient evaluations: 42
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.749289068624899
            Iterations: 42
            Function evaluations: 341
            Gradient evaluations: 42
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.749215959618139
            Iterations: 42
            Function evaluations: 341
            Gradient evaluations: 42
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.7491290293996045
            Iterations: 42
            Functio

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.748782798182194
            Iterations: 45
            Function evaluations: 365
            Gradient evaluations: 45
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.748754145004412
            Iterations: 45
            Function evaluations: 365
            Gradient evaluations: 45
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.748900892978499
            Iterations: 45
            Function evaluations: 365
            Gradient evaluations: 45
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.748801029986401
            Iterations: 45
            Function evaluations: 365
            Gradient evaluations: 45
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.748746895082606
            Iterations: 45
            Function

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.748194346193193
            Iterations: 40
            Function evaluations: 325
            Gradient evaluations: 40
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.74832111083588
            Iterations: 43
            Function evaluations: 349
            Gradient evaluations: 43
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.748252105176622
            Iterations: 43
            Function evaluations: 349
            Gradient evaluations: 43
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.748203925605677
            Iterations: 42
            Function evaluations: 341
            Gradient evaluations: 42
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.748257131080866
            Iterations: 43
            Function 

...

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.773146310854472
            Iterations: 30
            Function evaluations: 245
            Gradient evaluations: 30
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.773147992252777
            Iterations: 31
            Function evaluations: 253
            Gradient evaluations: 31
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.773329450015539
            Iterations: 33
            Function evaluations: 269
            Gradient evaluations: 33
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.773165324260232
            Iterations: 31
            Function evaluations: 255
            Gradient evaluations: 31
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.773290200179722
            Iterations: 31
            Function

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.773681707468513
            Iterations: 32
            Function evaluations: 264
            Gradient evaluations: 32
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.773816475474296
            Iterations: 33
            Function evaluations: 272
            Gradient evaluations: 33
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.773783293994764
            Iterations: 31
            Function evaluations: 256
            Gradient evaluations: 31
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.773840196975427
            Iterations: 31
            Function evaluations: 255
            Gradient evaluations: 31
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.7737964928605106
            Iterations: 31
            Functio

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.774407306811416
            Iterations: 32
            Function evaluations: 266
            Gradient evaluations: 32
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.774465312610029
            Iterations: 34
            Function evaluations: 279
            Gradient evaluations: 34
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.774384021895722
            Iterations: 33
            Function evaluations: 272
            Gradient evaluations: 33
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.774395990397547
            Iterations: 33
            Function evaluations: 272
            Gradient evaluations: 33
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.774412121023855
            Iterations: 29
            Function

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.774845555167865
            Iterations: 35
            Function evaluations: 291
            Gradient evaluations: 35
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.774851504280549
            Iterations: 36
            Function evaluations: 299
            Gradient evaluations: 36
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.774854701263748
            Iterations: 34
            Function evaluations: 281
            Gradient evaluations: 34
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.774885757388015
            Iterations: 35
            Function evaluations: 289
            Gradient evaluations: 35
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.774879286064013
            Iterations: 37
            Function

IndexError: index 500 is out of bounds for axis 1 with size 500

In [4]:
noise_matx

NameError: name 'noise_matx' is not defined

In [151]:
smooth_matx

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,0.619229,0.639381,0.632596,0.627576,0.621739,0.587789,0.581754,0.575493,0.600561,0.540163,...,0.049092,0.049046,0.049001,0.048956,0.048913,0.048869,0.048891,0.048847,0.048804,0.048762
1,0.628734,0.647048,0.640308,0.635249,0.629414,0.597417,0.591398,0.585167,0.608132,0.551522,...,0.042865,0.042821,0.042777,0.042733,0.042691,0.042648,0.042682,0.042640,0.042598,0.042557
2,0.638239,0.654715,0.648020,0.642922,0.637089,0.607046,0.601043,0.594840,0.615704,0.562880,...,0.036639,0.036595,0.036552,0.036510,0.036468,0.036427,0.036473,0.036432,0.036391,0.036352
3,0.647744,0.662381,0.655732,0.650595,0.644765,0.616675,0.610687,0.604514,0.623275,0.574239,...,0.030412,0.030369,0.030328,0.030286,0.030246,0.030206,0.030264,0.030224,0.030185,0.030146
4,0.657248,0.670048,0.663444,0.658268,0.652440,0.626303,0.620332,0.614187,0.630847,0.585598,...,0.024186,0.024144,0.024103,0.024063,0.024024,0.023985,0.024055,0.024016,0.023978,0.023941
5,0.666753,0.677715,0.671157,0.665941,0.660115,0.635932,0.629976,0.623861,0.638418,0.596956,...,0.017959,0.017918,0.017879,0.017839,0.017801,0.017764,0.017846,0.017809,0.017772,0.017736
6,0.676258,0.685382,0.678869,0.673614,0.667790,0.645561,0.639620,0.633535,0.645989,0.608315,...,0.011733,0.011693,0.011654,0.011616,0.011579,0.011543,0.011637,0.011601,0.011565,0.011531
7,0.685763,0.693049,0.686581,0.681287,0.675466,0.655189,0.649265,0.643208,0.653561,0.619674,...,0.005506,0.005467,0.005429,0.005392,0.005357,0.005321,0.005428,0.005393,0.005359,0.005325
8,0.695267,0.700716,0.694293,0.688960,0.683141,0.664818,0.658909,0.652882,0.661132,0.631033,...,-0.000506,-0.000544,-0.000581,-0.000617,-0.000652,-0.000686,-0.000568,-0.000602,-0.000635,-0.000667
9,0.704772,0.708382,0.702005,0.696633,0.690816,0.674447,0.668554,0.662555,0.668704,0.642391,...,-0.006096,-0.006134,-0.006170,-0.006206,-0.006240,-0.006273,-0.006145,-0.006178,-0.006211,-0.006242


* py-earth

In [5]:
def earth_smooth_matrix(nm_array,data_matrix,noise_coefficient):
    num_array = np.shape(data_matrix)[0]
    
    smooth_matx = pd.DataFrame(np.empty((num_array,1)), columns = ['a'])
    noise_matx = pd.DataFrame(np.empty((num_array,1)), columns = ['a'])
    
    for i in range(num_array):
        data_array = data_matrix[:, i]
        
        # get noise and smooth list
        noise_array = add_noise(nm_array, data_array, noise_coefficient).tolist()
        smooth_array =earth_Smoothing(nm_array,data_array,noise_coefficient).tolist()
        
        # get noise dataframe
        DF = pd.DataFrame(noise_array,columns = [i])
        noise_matx = noise_matx.join(DF)
        
        # get smooth dataframe
        df = pd.DataFrame(smooth_array,columns = [i])
        smooth_matx = smooth_matx.join(df)
        
    # drop the first columns  
    noise_matx = noise_matx.drop(columns='a')
    smooth_matx = smooth_matx.drop(columns='a')
        
    return noise_matx, smooth_matx

In [6]:
noisez_matx, smooth_matx = earth_smooth_matrix(datanm,dataz_matx,0.1)

NameError: name 'datanm' is not defined

In [137]:
noisez_matx

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,0.642958,0.637558,0.632158,0.626758,0.621458,0.616158,0.610958,0.605758,0.600558,0.595458,...,0.037439,0.037421,0.037403,0.037385,0.037368,0.037351,0.037335,0.037318,0.037302,0.037286
1,0.750254,0.744754,0.739254,0.733854,0.728454,0.723154,0.717754,0.712554,0.707354,0.702154,...,0.132600,0.132581,0.132563,0.132544,0.132526,0.132508,0.132490,0.132473,0.132456,0.132440
2,0.516295,0.510695,0.505195,0.499695,0.494295,0.488895,0.483495,0.478195,0.472895,0.467695,...,-0.113389,-0.113410,-0.113429,-0.113449,-0.113468,-0.113487,-0.113505,-0.113524,-0.113542,-0.113559
3,0.594156,0.588556,0.582956,0.577456,0.571956,0.566456,0.561056,0.555656,0.550356,0.545056,...,-0.047556,-0.047577,-0.047598,-0.047618,-0.047638,-0.047658,-0.047678,-0.047697,-0.047716,-0.047735
4,0.727119,0.721419,0.715819,0.710219,0.704619,0.699119,0.693619,0.688219,0.682819,0.677419,...,0.073384,0.073362,0.073340,0.073318,0.073297,0.073276,0.073255,0.073235,0.073214,0.073195
5,0.660581,0.654881,0.649181,0.643581,0.637981,0.632381,0.626881,0.621381,0.615881,0.610481,...,-0.005073,-0.005097,-0.005120,-0.005143,-0.005165,-0.005188,-0.005209,-0.005231,-0.005252,-0.005273
6,0.863220,0.857420,0.851720,0.846020,0.840320,0.834720,0.829220,0.823620,0.818120,0.812620,...,0.185650,0.185626,0.185601,0.185577,0.185553,0.185530,0.185507,0.185484,0.185462,0.185440
7,0.703915,0.698115,0.692315,0.686615,0.680915,0.675215,0.669615,0.664015,0.658515,0.652915,...,0.014535,0.014508,0.014483,0.014457,0.014432,0.014408,0.014383,0.014360,0.014336,0.014313
8,0.871805,0.865905,0.860105,0.854305,0.848605,0.842805,0.837205,0.831505,0.825905,0.820405,...,0.170618,0.170591,0.170564,0.170537,0.170511,0.170485,0.170459,0.170434,0.170409,0.170385
9,0.807060,0.801160,0.795360,0.789460,0.783760,0.777960,0.772260,0.766560,0.760960,0.755360,...,0.094271,0.094243,0.094214,0.094186,0.094158,0.094131,0.094104,0.094078,0.094052,0.094026


In [138]:
smooth_matx

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,0.619229,0.639381,0.632596,0.627576,0.621739,0.587789,0.581754,0.575493,0.600561,0.540163,...,0.049092,0.049046,0.049001,0.048956,0.048913,0.048869,0.048891,0.048847,0.048804,0.048762
1,0.628734,0.647048,0.640308,0.635249,0.629414,0.597417,0.591398,0.585167,0.608132,0.551522,...,0.042865,0.042821,0.042777,0.042733,0.042691,0.042648,0.042682,0.042640,0.042598,0.042557
2,0.638239,0.654715,0.648020,0.642922,0.637089,0.607046,0.601043,0.594840,0.615704,0.562880,...,0.036639,0.036595,0.036552,0.036510,0.036468,0.036427,0.036473,0.036432,0.036391,0.036352
3,0.647744,0.662381,0.655732,0.650595,0.644765,0.616675,0.610687,0.604514,0.623275,0.574239,...,0.030412,0.030369,0.030328,0.030286,0.030246,0.030206,0.030264,0.030224,0.030185,0.030146
4,0.657248,0.670048,0.663444,0.658268,0.652440,0.626303,0.620332,0.614187,0.630847,0.585598,...,0.024186,0.024144,0.024103,0.024063,0.024024,0.023985,0.024055,0.024016,0.023978,0.023941
5,0.666753,0.677715,0.671157,0.665941,0.660115,0.635932,0.629976,0.623861,0.638418,0.596956,...,0.017959,0.017918,0.017879,0.017839,0.017801,0.017764,0.017846,0.017809,0.017772,0.017736
6,0.676258,0.685382,0.678869,0.673614,0.667790,0.645561,0.639620,0.633535,0.645989,0.608315,...,0.011733,0.011693,0.011654,0.011616,0.011579,0.011543,0.011637,0.011601,0.011565,0.011531
7,0.685763,0.693049,0.686581,0.681287,0.675466,0.655189,0.649265,0.643208,0.653561,0.619674,...,0.005506,0.005467,0.005429,0.005392,0.005357,0.005321,0.005428,0.005393,0.005359,0.005325
8,0.695267,0.700716,0.694293,0.688960,0.683141,0.664818,0.658909,0.652882,0.661132,0.631033,...,-0.000506,-0.000544,-0.000581,-0.000617,-0.000652,-0.000686,-0.000568,-0.000602,-0.000635,-0.000667
9,0.704772,0.708382,0.702005,0.696633,0.690816,0.674447,0.668554,0.662555,0.668704,0.642391,...,-0.006096,-0.006134,-0.006170,-0.006206,-0.006240,-0.006273,-0.006145,-0.006178,-0.006211,-0.006242


# -------------------------------------------------------------------------------------------
# * Get peak dataframe

In [10]:
# load some data
def loaddata(data_filename):
    """load matrix data"""
    data = np.genfromtxt(data_filename, delimiter='\t')
    data_nm = data[1:,0]    #wavelength in nm
    data_time = data[0,1:]
    data_z = data[1:, 1:]

    return data_nm, data_time, data_z

In [11]:
# add noise
def add_noise(nm_array, y_array, noise_coefficient):
    # Add noise
    np.random.seed(1800)
    y_noise = noise_coefficient * np.random.normal(size=nm_array.size)
    y_proc = y_array + y_noise
    
    return y_proc

In [50]:
def astropy_smoothing(nm_array, timedelay, noise_coefficient,gg_init):
    # Generate fake data
    np.random.seed(42)
    ydata = timedelay + noise_coefficient*np.random.normal(size=nm_array.size)
    # Now to fit the data create a new superposition with initial
    # guesses for the parameters:
    fitter = fitting.SLSQPLSQFitter()
    gg_fit = fitter(gg_init, nm_array, ydata)
    return gg_fit(nm_array)

## astropy and peakutils 

In [8]:
def astropy_peak_matrix(nm_array,data_matrix,noise_coefficient,threshold, min_dist,gg_init):
    num_array = np.shape(data_matrix)[1]
    true_peak = []
    smooth_peak = []
    
    for i in range(num_array):
        data_array = data_matrix[:, i]
        noise_array = add_noise(nm_array, data_array, noise_coefficient)
        smooth_array = astropy_smoothing(nm_array, data_array,noise_coefficient,gg_init)
        
        # get true peak matrix
        indexes=findpeak(data_array, threshold, min_dist).tolist()
        true_peak.append(indexes)
        #get smooth peak matrix
        
        indexes1=findpeak(smooth_array, threshold, min_dist).tolist()
        smooth_peak.append(indexes1)
        
        # transfer to dataframe
        true_df=pd.DataFrame(true_peak)
        smooth_df=pd.DataFrame(smooth_peak)
    
    return true_df, smooth_df

In [19]:
def findpeak(data_z_array, threshold, min_dist):
    """find peaks and return indices of the peaks"""    
    peak_indices = peakutils.indexes(data_z_array, thres=threshold, min_dist=min_dist)
    
    return peak_indices

In [18]:
matx_filename = '20180418_twogaussian_spectralshfit.txt'
datanm, datatime, dataz_matx = loaddata(matx_filename)
g1 = models.Gaussian1D(1, 950, 30)
g2 = models.Gaussian1D(0.3, 1300, 100)
gg_init = g1+g2

In [19]:
# get the peak position dataframe of smooth data set
true_df1, smooth_df1 = astropy_peak_matrix(datanm, dataz_matx, 0.1, 0, 10,gg_init)

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.7495624261835365
            Iterations: 35
            Function evaluations: 286
            Gradient evaluations: 35
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.749416522632002
            Iterations: 26
            Function evaluations: 213
            Gradient evaluations: 26
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.749433287148435
            Iterations: 35
            Function evaluations: 286
            Gradient evaluations: 35
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.749375198981689
            Iterations: 36
            Function evaluations: 293
            Gradient evaluations: 36
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.749431472957076
            Iterations: 36
            Functio

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.749372560100794
            Iterations: 36
            Function evaluations: 293
            Gradient evaluations: 36
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.749372368294726
            Iterations: 36
            Function evaluations: 294
            Gradient evaluations: 36
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.749402007284543
            Iterations: 37
            Function evaluations: 301
            Gradient evaluations: 37
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.749120628475032
            Iterations: 37
            Function evaluations: 301
            Gradient evaluations: 37
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.749316680218414
            Iterations: 38
            Function

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.749181534395437
            Iterations: 41
            Function evaluations: 333
            Gradient evaluations: 41
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.749093908424613
            Iterations: 41
            Function evaluations: 333
            Gradient evaluations: 41
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.748982964647777
            Iterations: 42
            Function evaluations: 341
            Gradient evaluations: 42
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.749130093999115
            Iterations: 42
            Function evaluations: 341
            Gradient evaluations: 42
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.749289068624899
            Iterations: 42
            Function

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.7488884416510055
            Iterations: 45
            Function evaluations: 365
            Gradient evaluations: 45
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.7487997602008925
            Iterations: 45
            Function evaluations: 365
            Gradient evaluations: 45
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.748782798182194
            Iterations: 45
            Function evaluations: 365
            Gradient evaluations: 45
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.748754145004412
            Iterations: 45
            Function evaluations: 365
            Gradient evaluations: 45
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.748900892978499
            Iterations: 45
            Functi

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.748210239071977
            Iterations: 43
            Function evaluations: 349
            Gradient evaluations: 43
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.748194346193193
            Iterations: 40
            Function evaluations: 325
            Gradient evaluations: 40
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.74832111083588
            Iterations: 43
            Function evaluations: 349
            Gradient evaluations: 43
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.748252105176622
            Iterations: 43
            Function evaluations: 349
            Gradient evaluations: 43
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.748203925605677
            Iterations: 42
            Function 

...

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.755633165255419
            Iterations: 46
            Function evaluations: 373
            Gradient evaluations: 46
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.755515675303961
            Iterations: 46
            Function evaluations: 373
            Gradient evaluations: 46
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.755657509661916
            Iterations: 45
            Function evaluations: 365
            Gradient evaluations: 45
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.755724289984852
            Iterations: 47
            Function evaluations: 381
            Gradient evaluations: 47
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.755840911030926
            Iterations: 46
            Function

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.773114966154575
            Iterations: 29
            Function evaluations: 238
            Gradient evaluations: 29
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.77311412793755
            Iterations: 31
            Function evaluations: 254
            Gradient evaluations: 31
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.773318663207655
            Iterations: 31
            Function evaluations: 255
            Gradient evaluations: 31
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.773146310854472
            Iterations: 30
            Function evaluations: 245
            Gradient evaluations: 30
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.773147992252777
            Iterations: 31
            Function 

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.77387972918667
            Iterations: 32
            Function evaluations: 261
            Gradient evaluations: 32
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.773681707468513
            Iterations: 32
            Function evaluations: 264
            Gradient evaluations: 32
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.773816475474296
            Iterations: 33
            Function evaluations: 272
            Gradient evaluations: 33
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.773783293994764
            Iterations: 31
            Function evaluations: 256
            Gradient evaluations: 31
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.773840196975427
            Iterations: 31
            Function 

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.774356732235045
            Iterations: 31
            Function evaluations: 255
            Gradient evaluations: 31
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.774407306811416
            Iterations: 32
            Function evaluations: 266
            Gradient evaluations: 32
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.774465312610029
            Iterations: 34
            Function evaluations: 279
            Gradient evaluations: 34
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.774384021895722
            Iterations: 33
            Function evaluations: 272
            Gradient evaluations: 33
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.774395990397547
            Iterations: 33
            Function

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.774826078750889
            Iterations: 35
            Function evaluations: 290
            Gradient evaluations: 35
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.774841925358913
            Iterations: 35
            Function evaluations: 292
            Gradient evaluations: 35
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.774845555167865
            Iterations: 35
            Function evaluations: 291
            Gradient evaluations: 35
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.774851504280549
            Iterations: 36
            Function evaluations: 299
            Gradient evaluations: 36
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.774854701263748
            Iterations: 34
            Function

* true peak position in the two-gaussian_spectralshfit

In [20]:
true_df1

,0,1
0,50,400
1,50,400
2,50,400
3,50,400
4,51,400
5,51,400
6,51,400
7,51,400
8,52,400
9,52,400


* smooth peak position in the two-gaussian_spectralshfit

In [21]:
smooth_df1

,0,1
0,51,395.0
1,51,395.0
2,51,395.0
3,51,395.0
4,51,395.0
5,52,395.0
6,52,394.0
7,52,394.0
8,52,394.0
9,53,394.0


* py-earth and peakutils 

In [7]:
def earth_peak_matrix(nm_array,data_matrix,noise_coefficient,threshold, min_dist):
    num_array = np.shape(data_matrix)[1]
    
    true_peak = []
    smooth_peak = []
    
    for i in range(num_array):
        data_array = data_matrix[:, i]
        noise_array = add_noise(nm_array, data_array, noise_coefficient)
        smooth_array = Earth_Smoothing(nm_array, data_array,noise_coefficient)
        
        indexes=findpeak(data_array, threshold, min_dist).tolist()
        true_peak.append(indexes)
        
        indexes1=findpeak(smooth_array, threshold, min_dist).tolist()
        smooth_peak.append(indexes1)
                
        # transfer to dataframe
        true_df=pd.DataFrame(true_peak)
        smooth_df=pd.DataFrame(smooth_peak)
    
    return true_df, smooth_df

In [20]:
matx_filename = '20180418_twogaussian_spectralshfit.txt'
datanm, datatime, dataz_matx = loaddata(matx_filename)
## get the peak position dataframe of true data set
true_df, smooth_df = earth_peak_matrix(datanm, dataz_matx, 0.1, 0, 10)

/Users/demiliu/miniconda3/lib/python3.6/site-packages/sklearn_contrib_py_earth-0.1.0-py3.6-macosx-10.7-x86_64.egg/pyearth/earth.py:802: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  pruning_passer.run()
/Users/demiliu/miniconda3/lib/python3.6/site-packages/sklearn_contrib_py_earth-0.1.0-py3.6-macosx-10.7-x86_64.egg/pyearth/earth.py:1055: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  coef, resid = np.linalg.lstsq(B, weighted_y[:, i])[0:2]


In [21]:
true_df

,0,1
0,50,400
1,50,400
2,50,400
3,50,400
4,51,400
5,51,400
6,51,400
7,51,400
8,52,400
9,52,400


In [22]:
smooth_df

,0,1,2
0,50,402.0,NaN
1,53,393.0,NaN
2,54,393.0,NaN
3,54,394.0,NaN
4,53,198.0,392.0
5,55,389.0,NaN
6,55,389.0,NaN
7,54,389.0,NaN
8,54,389.0,NaN
9,55,213.0,389.0


In [139]:
dataz_matx.shape

(700, 500)

# Peak width and fwhm Dataframe

In [69]:
def peakchar(data_nm, data_z_array, peak_index):
    """find the peak width, and intensity"""
    num_peaks = len(peak_index)
    
    #array of peak height
    height = [data_z_array[idx] for idx in peak_index]
    
    #array of peak width
    half_height = [ht / 2 for ht in height]

    fwhm_idx_1 = np.empty_like(half_height)
    fwhm_idx_2 = np.empty_like(fwhm_idx_1)
    fwhm_nm_1 = np.empty_like(fwhm_idx_1)
    fwhm_nm_2 = np.empty_like(fwhm_idx_1)
    
    for i in range(num_peaks):
        #find the index and nmof the left side of the fwhm
        if i == 0:
            fwhm_idx_1[i] = find_nearest(data_z_array[0:peak_index[i]], half_height[i])
        else:
            fwhm_idx_1[i] = find_nearest(data_z_array[peak_index[i-1]:peak_index[i]], half_height[i]) + peak_index[i-1]

        fwhm_nm_1[i] = data_nm[int(fwhm_idx_1[i])]
        
        #find the index and nm of the right side of the fwhm   
        fwhm_idx_2[i] = find_nearest(data_z_array[peak_index[i]:], half_height[i]) + peak_index[i]

        fwhm_nm_2[i] = data_nm[int(fwhm_idx_2[i])]
    
    #find fwhm
    fwhm = fwhm_nm_2 - fwhm_nm_1

    return height, fwhm

In [ ]:
def earth_peak_matrix(nm_array,data_matrix,noise_coefficient,threshold, min_dist):
    num_array = np.shape(data_matrix)[1]
    
    true_peak = []
    smooth_peak = []
    
    for i in range(num_array):
        data_array = data_matrix[:, i]
        noise_array = add_noise(nm_array, data_array, noise_coefficient)
        smooth_array = Earth_Smoothing(nm_array, data_array,noise_coefficient)
        
        indexes=findpeak(data_array, threshold, min_dist).tolist()
        true_peak.append(indexes)
        
        indexes1=findpeak(smooth_array, threshold, min_dist)
        smooth_peak.append(indexes1)
                
        # transfer to dataframe
        true_df=pd.DataFrame(true_peak)
        smooth_df=pd.DataFrame(smooth_peak)
    
    return true_df, smooth_df

In [343]:
def peak_matrix(nm_array,data_matrix, threshold, mindist):
    """find peaks in a data matrix"""
    peak_idx_matx = []
    peak_height_matx = []
    peak_fwhm_matx = []
    
    for i in range(num_timeslice):
        data_timeslice = data_matrix[:, i]
        
        peak_idx = findpeak(data_timeslice, threshold, mindist).tolist()
        peak_idx_matx.append(peak_idx)
        
        
        peak_height, peak_fwhm = peakchar(nm_array, data_timeslice, peak_idx)
        
        peak_height_matx.append(peak_height)
        peak_fwhm_matx.append(peak_fwhm)
        
        # transfer to dataframe
        peak_idx_df=pd.DataFrame(peak_idx_matx)
        peak_height_df=pd.DataFrame(peak_height_matx)
        peak_fwhm_df=pd.DataFrame(peak_fwhm_matx)
        
    return peak_idx_df, peak_height_df, peak_fwhm_df

In [344]:
def find_nearest(array,value):
    idx = (np.abs(array-value)).argmin()
    return idx

In [347]:
peak_idx_df, peak_height_df, peak_fwhm_df = peak_matrix(datanm,smooth_matx, 0.00, 50)

In [348]:
peak_height_df

,0,1
0,0.985442,0.302389
1,0.980625,0.299123
2,0.975647,0.296452
3,0.970760,0.293546
4,0.965868,0.290652
5,0.961060,0.287789
6,0.956346,0.284650
7,0.951495,0.282151
8,0.946728,0.279377
9,0.941967,0.276632


In [349]:
peak_fwhm_df

,0,1
0,110.0,233.0
1,110.0,233.0
2,111.0,233.0
3,111.0,233.0
4,111.0,233.0
5,111.0,233.0
6,111.0,233.0
7,112.0,233.0
8,112.0,233.0
9,112.0,233.0


In [350]:
peak_idx_df

,0,1
0,51,395.0
1,51,395.0
2,51,395.0
3,51,395.0
4,51,395.0
5,52,395.0
6,52,394.0
7,52,394.0
8,52,394.0
9,53,394.0


# --------------------------------------------------------